<a href="https://colab.research.google.com/github/Maleshhussen35/traffic-sign-/blob/main/speech_recogtion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

first we installes all required libarries


In [ ]:
!pip install sounddevice scipy transformers

In [ ]:
!apt-get install -qq libportaudio2

Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1.1) ...
Setting up libportaudio2:amd64 (19.6.0-1.1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.8) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_opencl.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm_debug.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_loader.so.0 is not a symbolic link

/sbin/ldconfig.real: 

In [ ]:
!pip install sounddevice

In [ ]:
import sounddevice as sd
print(sd.query_devices())  # List available audio devices

In [ ]:
from IPython.display import Javascript, Audio
from google.colab import output  # Corrected import
import numpy as np
from base64 import b64decode

# JavaScript to record audio in the browser
RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record_audio(duration=5, sample_rate=16000):
    display(Javascript(RECORD))
    data = output.eval_js(f'record({duration * 1000})')
    binary = b64decode(data.split(',')[1])
    audio = np.frombuffer(binary, dtype=np.int16)
    return audio, sample_rate

# Record audio
print("Recording... Speak now!")
audio, sample_rate = record_audio()
print("Recording complete.")

Recording... Speak now!


<IPython.core.display.Javascript object>

saving the recoredd if needed


In [ ]:
import scipy.io.wavfile as wav

wav.write("output.wav", sample_rate, audio)
print("Audio saved as output.wav")

using pretained speech recongnition

In [ ]:
from transformers import pipeline

# Load the Whisper model
speech_to_text = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Convert the recorded audio to text
audio = audio.astype(np.float32) / 32768.0  # Normalize audio data
transcription = speech_to_text(audio, sampling_rate=sample_rate)
print("Transcription:", transcription['text'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Device set to use cpu


NameError: name 'audio' is not defined

to make the system to work in real time


In [ ]:
while True:
    print("Recording... Speak now!")
    audio, sample_rate = record_audio(duration=5)
    print("Recording complete.")

    # Convert the recorded audio to text
    audio = audio.astype(np.float32) / 32768.0
    transcription = speech_to_text(audio, sampling_rate=sample_rate)
    print("You said:", transcription['text'])